<a href="https://colab.research.google.com/github/swarnava-96/Keras-Tuner/blob/main/Optimizing_CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Creating a CNN model and Optimizing it using Keras Tuner**

### Let's install Keras Tuner

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 96 kB 2.9 MB/s 


### Importing the necessary libraries and loading the dataset

In [2]:
# Importing the required libraries

import tensorflow as tf
from tensorflow import keras 
import numpy as np

In [3]:
# Let's check the version of tensorflow as keras tuner requires tensorflow version > 2.0

print(tf.__version__)

2.6.0


In [4]:
# Loading the dataset

fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


### Resizing the images



In [5]:
train_images = train_images/255.0
test_images = test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

### Hyperparameter Optimization

In [8]:
# Let's build our own custom function

def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters = hp.Int("conv_1_filter", min_value = 32, max_value = 128, step = 16),
        kernel_size = hp.Choice("conv_1_kernel", values = [3,5]),
        activation = "relu",
        input_shape = (28, 28, 1)
    ),
    keras.layers.Conv2D(
        filters = hp.Int("conv_2_filter", min_value = 32, max_value = 64, step = 16),
        kernel_size = hp.Choice("conv_2_kernel", values = [3,5]),
        activation = "relu"
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units = hp.Int('dense_1_units', min_value = 32, max_value = 128, step = 16),
        activation = "relu"
    ),
    keras.layers.Dense(10, activation = 'softmax')
  ])
  model.compile(optimizer = keras.optimizers.Adam(hp.Choice("learning_rate", values = [1e-2,1e-3])),
                loss = "sparse_categorical_crossentropy",
                metrics = ["accuracy"])
  return model

In [9]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [10]:
tuner_search = RandomSearch(build_model,
                            objective = "val_accuracy",
                            max_trials = 5, directory = "output", project_name = "Mnist Fashion")

In [11]:
# Fitting the model

tuner_search.search(train_images, train_labels, epochs = 3, validation_split = 0.1)

Trial 5 Complete [00h 00m 50s]
val_accuracy: 0.9086666703224182

Best val_accuracy So Far: 0.9139999747276306
Total elapsed time: 00h 05m 45s
INFO:tensorflow:Oracle triggered exit


In [12]:
model = tuner_search.get_best_models(num_models = 1)[0]

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        32288     
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1179712   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 1,213,770
Trainable params: 1,213,770
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Fitting the model with the best parameters

model.fit(train_images, train_labels, epochs = 10, validation_split = 0.1, initial_epoch = 3)

Epoch 4/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.1546 - accuracy: 0.9431 - val_loss: 0.2603 - val_accuracy: 0.9150
Epoch 5/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.1134 - accuracy: 0.9584 - val_loss: 0.3050 - val_accuracy: 0.9123
Epoch 6/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0835 - accuracy: 0.9689 - val_loss: 0.3258 - val_accuracy: 0.9107
Epoch 7/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0614 - accuracy: 0.9773 - val_loss: 0.3569 - val_accuracy: 0.9098
Epoch 8/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0458 - accuracy: 0.9832 - val_loss: 0.3918 - val_accuracy: 0.9120
Epoch 9/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0373 - accuracy: 0.9871 - val_loss: 0.4145 - val_accuracy: 0.9148
Epoch 10/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0300 - accuracy: 0.9893 - val_loss: 0.4663 - val_a